In [4]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import requests
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from datetime import date, timedelta
import numpy as np
import joblib

# --- CONFIGURAZIONE ---
FILE_NAME = 'report_unito_ordinato.csv'
LATITUDE = 44.77   # Correggio
LONGITUDE = 10.78  # Correggio

# 1. Caricamento dati produzione
print("Caricamento dati produzione...")
df_energy = pd.read_csv(FILE_NAME)
df_energy['Date'] = pd.to_datetime(df_energy['Date'])

print(df_energy.head())

# 1. Definisci le colonne che *vuoi* mantenere
colonne_da_mantenere = ['Date', 'Daily Production (Active)']

# 2. Identifica le colonne che *non* vuoi mantenere
colonne_da_rimuovere = [
    col for col in df_energy.columns
    if col not in colonne_da_mantenere
]

# 3. Rimuovi le colonne indesiderate dal DataFrame
# axis=1 specifica che l'operazione è sulle colonne
df_energy.drop(columns=colonne_da_rimuovere, inplace=True)

print(f"Rimosse {len(colonne_da_rimuovere)} colonne.")
print(f"Colonnes rimaste: {df_energy.columns.tolist()}")
print(df_energy.head())

# Otteniamo il range di date dal file
start_date = df_energy['Date'].min().strftime('%Y-%m-%d')
end_date = df_energy['Date'].max().strftime('%Y-%m-%d')
print(f"Periodo individuato: {start_date} - {end_date}")

# 2. Download dati meteo (Open-Meteo API)
print("Scaricamento dati meteo da Open-Meteo...")
# Usiamo l'endpoint 'archive' per dati storici o 'forecast' con past_days se molto recenti.
# Qui usiamo la richiesta standard che copre il range date.
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": LATITUDE,
    "longitude": LONGITUDE,
    "start_date": start_date,
    "end_date": end_date,
    # Variabili chiave per il fotovoltaico: Radiazione Solare, Temperatura (Max/Min), Precipitazioni
    "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,shortwave_radiation_sum",
    "timezone": "auto"
}

response = requests.get(url, params=params)
if response.status_code != 200:
    raise Exception("Errore nel scaricare i dati meteo: " + response.text)

weather_json = response.json()

# Creazione DataFrame Meteo
daily_data = weather_json['daily']
df_weather = pd.DataFrame({
    'Date': pd.to_datetime(daily_data['time']),
    'solar_radiation': daily_data['shortwave_radiation_sum'], # Fattore più importante
    'temp_max': daily_data['temperature_2m_max'],
    'temp_min': daily_data['temperature_2m_min'],
    'precipitation': daily_data['precipitation_sum']
})

# 3. Unione dei dati (Merge)
print("Unione dataset...")
df_final = pd.merge(df_energy, df_weather, on='Date', how='inner')

# Rimuoviamo eventuali righe con dati mancanti
df_final = df_final.dropna()
# Assumendo che il nome della colonna sia 'Daily'
indici_massimi = df_final.groupby('Date')['Daily Production (Active)'].idxmax()

# Seleziona le righe corrispondenti agli indici massimi
df = df_final.loc[indici_massimi]

# 4. Preparazione Machine Learning
# X = Features (Meteo), y = Target (Produzione)

X = df[['solar_radiation', 'temp_max', 'temp_min', 'precipitation']]
y = df['Daily Production (Active)']
df.head(5)


# --- CONFIGURAZIONE ---
FILE_DATI = 'report_unito_ordinato.csv' # File Excel
LATITUDE = 44.77   # Correggio
LONGITUDE = 10.78  # Correggio

# Aggiungiamo le nuove variabili qui
METEO_PARAMS = (
    "temperature_2m_max,temperature_2m_min,"
    "precipitation_sum,shortwave_radiation_sum,"
    "wind_speed_10m_max,cloud_cover_mean,daylight_duration,snowfall_sum"
)

def get_weather_data(start_date, end_date, is_forecast=False):
    """Scarica dati meteo estesi da Open-Meteo"""
    url = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": LATITUDE,
        "longitude": LONGITUDE,
        "start_date": start_date,
        "end_date": end_date,
        "daily": METEO_PARAMS,
        "timezone": "auto"
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Errore API Meteo: {response.text}")

    data = response.json()['daily']

    df = pd.DataFrame({
        'Date': pd.to_datetime(data['time']),
        'solar_radiation': data['shortwave_radiation_sum'],
        'temp_max': data['temperature_2m_max'],
        'temp_min': data['temperature_2m_min'],
        'precipitation': data['precipitation_sum'],
        # Nuove Feature
        'wind_speed': data['wind_speed_10m_max'],
        'cloud_cover': data['cloud_cover_mean'],
        'daylight_duration': data['daylight_duration'],
        'snowfall': data['snowfall_sum']
    })
    # Gestione valori nulli (es. se manca il dato neve, mettiamo 0)
    return df.fillna(0)

def main():
    print("--- 1. ADDESTRAMENTO DEL MODELLO POTENZIATO ---")

    # 1. Carica, Pulisci e Aggrega Dati Energia
    try:
        # Uso pd.read_excel come hai specificato
        df_energy = pd.read_csv(FILE_DATI)
        df_energy['Date'] = pd.to_datetime(df_energy['Date'])
    except FileNotFoundError:
        print(f"Errore: File '{FILE_DATI}' non trovato.")
        return
    except ValueError as e:
        print(f"Errore nella lettura del file o colonna Date: {e}. Controlla la colonna 'Date'.")
        return

    # a) Seleziona solo le colonne necessarie
    # Questo garantisce che si usino solo Date e la produzione, scartando tutte le altre
    colonne_da_mantenere = ['Date', 'Daily Production (Active)']
    df_energy = df_energy[colonne_da_mantenere]

    # b) MANTIENI LA RIGA CON LA PRODUZIONE MASSIMA PER OGNI DATA
    # Questo aggrega i dati a livello giornaliero, essenziale per il merge con i dati meteo giornalieri
    if 'Daily Production (Active)' in df_energy.columns:
        print("Aggregazione: Mantengo la produzione massima per ogni giorno...")
        indici_massimi = df_energy.groupby('Date')['Daily Production (Active)'].idxmax()
        df_energy = df_energy.loc[indici_massimi]

    # c) Rimuovi eventuali NaN residui dopo l'aggregazione
    df_energy.dropna(inplace=True)
    print(f"Dati di addestramento pronti, {len(df_energy)} giorni di dati.")

    # 2. Scarica meteo storico
    start_hist = df_energy['Date'].min().strftime('%Y-%m-%d')
    end_hist = df_energy['Date'].max().strftime('%Y-%m-%d')
    print(f"Recupero dati meteo estesi ({start_hist} - {end_hist})...")

    df_weather_hist = get_weather_data(start_hist, end_hist)

    # 3. Unisci e Prepara
    # Non serve dropna() dopo il merge perché i dati sono stati puliti prima e il merge è inner.
    df_train = pd.merge(df_energy, df_weather_hist, on='Date')

    # Lista aggiornata delle features
    feature_cols = [
        'solar_radiation', 'temp_max', 'temp_min', 'precipitation',
        'wind_speed', 'cloud_cover', 'daylight_duration', 'snowfall'
    ]

    X_train = df_train[feature_cols]
    y_train = df_train['Daily Production (Active)']

    # 4. Allena Random Forest
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Mostriamo quali variabili il modello ha trovato più utili
    print("\n--- IMPORTANZA VARIABILI (Top 5) ---")
    importances = pd.Series(model.feature_importances_, index=feature_cols)
    print(importances.sort_values(ascending=False).head(5))

    print("\n--- 2. PREVISIONE PER DOMANI ---")
    today = date.today()
    tomorrow = today + timedelta(days=1)
    tomorrow_str = tomorrow.strftime('%Y-%m-%d')

    print(f"Scarico previsioni per domani ({tomorrow_str})...")
    df_forecast = get_weather_data(tomorrow_str, tomorrow_str, is_forecast=True)

    X_tomorrow = df_forecast[feature_cols]
    prediction = model.predict(X_tomorrow)[0]

    meteo = X_tomorrow.iloc[0]
    print("-" * 40)
    print(f"DATA: {tomorrow_str}")
    print(f"Scenario Meteo:")
    # Formattazione per una migliore leggibilità
    print(f"  ☀️ Radiazione: {meteo['solar_radiation']:.2f} MJ/m²")
    print(f"  ☁️ Nuvolosità: {meteo['cloud_cover']:.1f}%")
    print(f"  💨 Vento Max: {meteo['wind_speed']:.1f} km/h")
    print(f"  💧 Pioggia: {meteo['precipitation']:.2f} mm")
    print(f"  🌡️ Temp Max/Min: {meteo['temp_min']:.1f}°C / {meteo['temp_max']:.1f}°C")
    print(f"  ⏳ Durata Luce: {meteo['daylight_duration']/3600:.1f} ore") # Convertito in ore
    if meteo['snowfall'] > 0:
        print(f"  ❄️ Neve: {meteo['snowfall']:.2f} cm")

    print("-" * 40)
    print(f"⚡ PRODUZIONE STIMATA: {prediction:.2f} kWh")
    print("-" * 40)

    nome_file_modello = 'modello_previsione_produzione.joblib'
    joblib.dump(model, nome_file_modello)

if __name__ == "__main__":
    main()

Caricamento dati produzione...
        Date                Serial            Timestamp  Timezone  \
0 2025-09-29  ZH1025006KE24A1700R7  2025-09-29 11:45:51         8   
1 2025-09-29  ZH1025006KE24A1700R7  2025-09-29 11:46:19         8   
2 2025-09-29  ZH1025006KE24A1700R7  2025-09-29 11:52:02         8   
3 2025-09-29  ZH1025006KE24A1700R7  2025-09-29 11:57:06         8   
4 2025-09-29  ZH1025006KE24A1700R7  2025-09-29 12:02:09         8   

   AC Voltage R  AC Current R  Grid Frequency  Power Battery Pack 1  \
0         237.7          0.02           49.99                   230   
1         237.7          0.02           49.99                   230   
2         237.8          1.30           50.03                    20   
3         239.4          1.38           49.99                  1710   
4         236.8          0.86           50.02                  1170   

   Power Battery Pack 2  Total Grid Power  ...  Daily Consumption  \
0                     0                 0  ...            